# Analyse failed trades

Analyse failed trades over multiple trade executors.

 - Show failed and repaired and fix trades
  - Analyse slippage parameters and other causes of failure



In [1]:
monitored_executors = [
    #"https://enzyme-polygon-eth-usdc.tradingstrategy.ai",
    "https://enzyme-polygon-eth-usdc-sls.tradingstrategy.ai",
    #"https://enzyme-polygon-matic-usdc.tradingstrategy.ai",
    # "https://enzyme-polygon-multipair.tradingstrategy.ai",
]



In [2]:
import requests
from tradeexecutor.state.state import State

total_downloaded = 0

def download_state(url) -> State:
    global total_downloaded
    resp = requests.get(f"{url}/state")

    if len(resp.content) == 0:
        raise RuntimeError(f"Could not download: {url}")

    try:
        state = State.read_json_blob(resp.text)
    except Exception as e:
        raise RuntimeError(f"Could not decode: {url}") from e
    total_downloaded += len(resp.content)
    return state

states = [download_state(url)for url in monitored_executors]


print(f"Downloaded {total_downloaded:,} bytes")

Downloaded 105,009 bytes


In [3]:
from tradeexecutor.state.trade import TradeExecution
from itertools import chain

# Get all trades across different trade executors
all_trades = chain(*[state.portfolio.get_all_trades() for state in states])

t: TradeExecution
failed_trades = [t for t in all_trades if t.is_failed() or t.is_repaired() or t.is_repair_trade()]

print(f"Total {len(failed_trades)} failed and repaired trades")

Total 1 failed and repaired trades


In [4]:
import pandas as pd
from tradeexecutor.analysis.slippage import display_slippage


pd.set_option('display.max_colwidth', None)

df = display_slippage(failed_trades)
display(df)

,Flags,Position,Time,Trade,Lag,Slippage,amountIn,amountO ut,Enzyme expected amount,Assumed price,Uniswap price,Failure reason,Tx
1,FAIL,#1,2023-07-15 15:02:50,USDC->WETH,159.0,100,8.589988,0.004445456425600057,0.004401151327881396,1932.048574,1932.307321815780809367215429,Too little received                                        ,0x842d5d7fdb89007e591d79a3b65c938ee97fccfa1b638fa172cd38eb110eb314
